# <div align="center" style="color: #ff5733;">Sil Account level tagging based on Item Purchased</div>

# Logic for this is to select the data based on the Item purchased. If the Item Purchased in Mobile phone then it is a Mobile store

Created by : Dwaipayan Chakroborti<br>
Requestor: Anastasia -- Refer to her group chat with Heaven, Biswa, Me and Sreekanth<br>
Dated: 2024-09-30

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """     
SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  lmt.new_loan_type,
  lmt.flagDisbursement,
  lmt.disbursementDateTime,
  lmt.disbursedLoanAmount,
  lmt.downPaymentAmount,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
  `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf
ON
  UPPER(asf.PURPLE_KEY) = UPPER(lmt.purpleKey) --
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.flagDisbursement = 1
 and lmt.disbursementDateTime is not null
  ;
"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Query is running:   0%|          |

NotFound: 404 Not found: Table prj-prod-dataplatform:worktable_datachampions.AREA_SPLIT_FINAL was not found in location asia-southeast1; reason: notFound, message: Not found: Table prj-prod-dataplatform:worktable_datachampions.AREA_SPLIT_FINAL was not found in location asia-southeast1

Location: asia-southeast1
Job ID: 2e6bff7e-b736-4a70-af97-d06434541db2


In [6]:
df.to_csv(r"LoanLevelPurchasedITEM.csv", index = False)

# New query with the code part from Marek also included for better clarity

In [4]:
sq = """SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  substr(cast(date_trunc(lmt.decision_date, MONTH) as string),0,10) as mth,
  lmt.employmentstatus,
  case
  when EXTRACT(HOUR FROM lmt.termsAndConditionsSubmitDateTime)<18 then "<18"
  else "18+" end as application_hour,
  case
  when upper(lmt.purposeDescription) like "%MOBILE%" then "mobile"
  else "non-mobile" end as goods_type,
  case
  when c.store_type=1 then "appliance"
  else "mobile" end as pos_type,
  c.mer_name as retailer,
  lmt.new_loan_type,
  lmt.loantype,
  lmt.flagApproval as approved,
  lmt.flagDisbursement as disbursed,
  lmt.flagrejection as rejected,
  lmt.disbursedloanamount as volume_out,
  lmt.disbursedLoanTenur as tenor,
  lmt.disbursementDateTime,
  lmt.downPaymentAmount,
  case
    when lmt.loanRequestAmount<=10000 then "<10k"
    when lmt.loanRequestAmount<=15000 then "<15k"
    when lmt.loanRequestAmount<=20000 then "<20k"
    when lmt.loanRequestAmount<=25000 then "<25k"
    when lmt.loanRequestAmount<=30000 then "<30k"
    else "30k+" end as ca_category,
  case
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.195 then "<20%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.295 then "<30%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.35 then "<35%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.4 then "<40%"
  else "40+" end as DP_range,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.mer_category STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_CURRENTV2 asf
ON
 UPPER(asf.purpleKey) = UPPER(lmt.purpleKey) --
inner join `dl_loans_db_raw.tdbk_merchant_refferal_mtb` c on lmt.purpleKey=c.mer_refferal_code
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.loantype="BNPL"
  ;"""

In [5]:
dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 3d4c531f-55e0-46df-85b2-2d0cb7c48f5a successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [6]:
dfd.shape

(375523, 35)

In [7]:
dfd.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'purpleKey',
       'merchantName', 'startApplyDateTime', 'applicationStatus',
       'decision_date', 'mth', 'employmentstatus', 'application_hour',
       'goods_type', 'pos_type', 'retailer', 'new_loan_type', 'loantype',
       'approved', 'disbursed', 'rejected', 'volume_out', 'tenor',
       'disbursementDateTime', 'downPaymentAmount', 'ca_category', 'DP_range',
       'loanCategoryId', 'createdDate', 'brandName', 'skuNo', 'price',
       'product_id', 'product_category', 'product_description', 'STORE_TAG',
       'product_downpayment', 'product_crifcode'],
      dtype='object')

In [8]:
pd.set_option("Display.max_columns", None)
dfd.head()

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,goods_type,pos_type,retailer,new_loan_type,loantype,approved,disbursed,rejected,volume_out,tenor,disbursementDateTime,downPaymentAmount,ca_category,DP_range,loanCategoryId,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
0,cfc1a262-029b-4762-87a2-e1ee3f418ef4,None,MU7HCT0,FC Home Center Sapang Palay 2,2023-08-05 17:02:48,RESET,NaT,None,SELF EMPLOYED,18+,non-mobile,appliance,FC Home Center Sapang Palay 2,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,5000.00,<20k,<30%,1723,2023-08-05 09:02:41+00:00,Haier,HWM100-B1678B,22299.00,1723,None,"Washing machine, dryer or both",Appliances Retailer,0.0,120
1,57740369-0558-438f-9e3d-d7e2195b9931,None,M2X80GJ,FC Home Center Karuhatan 1,2023-12-08 12:12:37,RESET,NaT,None,SALARIED,18+,non-mobile,appliance,FC Home Center Karuhatan 1,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,5000.00,<10k,40+,1721,2023-12-08 04:12:23+00:00,Haier,HR-IV168,12199.00,1721,None,Refrigerators,Appliances Retailer,0.0,119
2,21f3cd6e-05b2-44f2-8da8-9d5d159c42d5,None,MU1A9P4,Home Along Los Banos,2024-06-22 17:53:58,RESET,NaT,None,SALARIED,18+,non-mobile,appliance,Home Along Los Banos,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,1000.00,<10k,<20%,1719,2024-06-22 09:53:41+00:00,astron,gme.led-3277 smartplus,7999.00,1719,None,Televisions,Home Appliances Retailer,0.0,116
3,1b047f2c-ffcf-4a3a-85d2-855a948f1350,None,M2O6RBI,Home Along Binakayan,2024-09-26 15:35:43,RESET,NaT,None,HOUSEWIFE/HOUSE HUSBAND,18+,non-mobile,appliance,Home Along Binakayan,SIL ZERO,BNPL,0,0,0,NaN,<NA>,NaT,4000.00,<15k,<30%,1719,2024-09-26 07:35:14+00:00,Prestiz,One.wpreu4303,17399.00,1719,None,Televisions,Home Appliances Retailer,0.0,116
4,b7b9d774-4b0b-44ef-94cc-9e969258496c,None,MD1R4BJ,FC Home Center Naic 2,2023-11-06 12:34:47,RESET,NaT,None,BUSINESS OWNER/SOLE PROPRIETOR,18+,non-mobile,appliance,FC Home Center Naic 2,SIL-Instore,BNPL,0,0,0,NaN,<NA>,NaT,2800.00,<15k,<30%,1726,2023-11-06 04:06:55+00:00,Megaboom,DS C121 6S,13679.00,1726,None,"Dining set, Chairs or Tables",Appliances Retailer,0.0,122


In [9]:
print(dfd.dtypes)

digitalLoanAccountId                 object
loanAccountNumber                    object
purpleKey                            object
merchantName                         object
startApplyDateTime           datetime64[us]
applicationStatus                    object
decision_date                datetime64[us]
mth                                  object
employmentstatus                     object
application_hour                     object
goods_type                           object
pos_type                             object
retailer                             object
new_loan_type                        object
loantype                             object
approved                              Int64
disbursed                             Int64
rejected                              Int64
volume_out                          float64
tenor                                 Int64
disbursementDateTime         datetime64[us]
downPaymentAmount                    object
ca_category                     

In [10]:
datetime_columns = ['startApplyDateTime', 'decision_date', 'disbursementDateTime', 'createdDate']

# Remove timezone information
for col in datetime_columns:
    dfd[col] = dfd[col].dt.tz_localize(None)
    


In [11]:
dfd.to_excel("Sil_DisbursedData_20241202.xlsx", index = False)